In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from scipy import stats

In [ ]:
def get_treatment(
    t_a=5,
    t_b=5,
    c_a=5,
    c_b=5,
    idx=None,
    prob_history=None,
    t_ctr_history=None,
    c_ctr_history=None,
):
    treatment = stats.beta(t_a, t_b)
    control = stats.beta(c_a, c_b)
    t_sample = treatment.rvs(10_000)
    c_sample = control.rvs(10_000)
    x = np.linspace(0, 1, 1000)
    t_pdf = treatment.pdf(x)
    c_pdf = control.pdf(x)

    t_ctr = t_a / (t_a + t_b)
    c_ctr = c_a / (c_a + c_b)

    prob_better = np.mean(t_sample > c_sample)
    lift = np.mean(t_sample - c_sample)

    plt.subplot(1, 2, 1)
    plt.hist(
        t_sample, bins=50, label="treatment", alpha=0.4, density=True, color="magenta"
    )
    plt.hist(c_sample, bins=50, label="control", alpha=0.4, density=True, color="blue")
    plt.plot(x, t_pdf, color="magenta")
    plt.plot(x, c_pdf, color="blue")
    plt.title("CTR posterior distribution")

    plt.suptitle(
        f"probability treatment better: {prob_better:.1%} lift: {lift:.1%}, treatment "
        f"CTR: {t_ctr:.1%}, control CTR: {c_ctr:.1%} sample size: {idx:,}"
    )
    plt.xlim(0, 1)

    if idx is None or max(t_pdf) < 9:
        ymax = 9
    elif max(t_pdf) < 18:
        ymax = 18
    plt.ylim(0, ymax)
    plt.legend()

    if idx is not None:
        plt.subplot(1, 2, 2)
        x_history = range(idx + 1)
        plt.plot(
            x_history,
            prob_history[: idx + 1],
            label="Prob Treatment Better",
            color="green",
        )
        plt.plot(
            x_history, t_ctr_history[: idx + 1], label="Treatment CTR", color="magenta"
        )
        plt.plot(x_history, c_ctr_history[: idx + 1], label="Control CTR", color="blue")
        plt.title("Metrics Over Time")
        plt.xlabel("Sample Size")
        plt.ylabel("Probability")
        plt.legend()
        plt.grid(True)
    plt.tight_layout()


t_as = (np.random.random(1000) < 0.6).cumsum()
t_bs = np.arange(1000) - t_as + 1
c_as = (np.random.random(1000) < 0.5).cumsum()
c_bs = np.arange(1000) - c_as + 1
t_as = t_as + 1
t_bs = t_bs + 1
c_as = c_as + 1
c_bs = c_bs + 1
assert t_as[0] + t_bs[0] == 3
assert c_as[0] + c_bs[0] == 3

prob_history = np.zeros(1000)
t_ctr_history = np.zeros(1000)
c_ctr_history = np.zeros(1000)

for i in range(1000):
    treatment = stats.beta(t_as[i], t_bs[i])
    control = stats.beta(c_as[i], c_bs[i])
    t_sample = treatment.rvs(10_000)
    c_sample = control.rvs(10_000)
    prob_history[i] = np.mean(t_sample > c_sample)
    t_ctr_history[i] = t_as[i] / (t_as[i] + t_bs[i])
    c_ctr_history[i] = c_as[i] / (c_as[i] + c_bs[i])

fig = plt.figure(figsize=(20, 6))


def animate(frame):
    plt.clf()
    get_treatment(
        t_as[frame],
        t_bs[frame],
        c_as[frame],
        c_bs[frame],
        frame,
        prob_history,
        t_ctr_history,
        c_ctr_history,
    )


anim = FuncAnimation(fig, animate, frames=100, interval=100)
anim.save("bayesian_ab_test.gif", writer="pillow")

In [ ]:
# interact(get_treatment,
#         t_a=(1, 100, 1),
#         t_b=(1, 100, 1),
#         c_a=(1, 100, 1),
#         c_b=(1, 100, 1)
# )